In [ ]:
#! pip install -q --upgrade langchain langchain-openai langchain-core langchain_community docx2txt pypdf  langchain_chroma sentence_transformers

### **Part 1:**

- **Understanding RAG (Retrieval-Augmented Generation)**
    - What is RAG?
    - Importance of RAG in AI applications
- **Introduction to LangChain and LCEL**
    - Overview of LangChain
    - LangChain Expression Language (LCEL)
- **Exploring LangChain Components**
    - LLM (Large Language Models)
    - Prompts
    - Retrievers
    - Composing components into a chain
- **Document Processing and Vector Databases**
    - Document splitting techniques
    - Embedding documents
    - Storing and retrieving documents in a vector database

- **Creating Your First RAG Chain**
    - Step-by-step guide to building a basic RAG chain
    - Answering questions from documents using RAG
- **Building a Conversational RAG**
    - Handling follow-up questions
    - Concept of contextualizing and refining queries
- **Using pre-built LangChain RAG chains**
    - history_aware_retriever
    - create_retrieval_chain
- **Building Multi User Chatbot**
    - Managing conversation history using a database table

### **Part 2: Moving to Production with FastAPI**

- **Integrating Colab Code with FastAPI**
    - Setting up FastAPI for production
    - Modularizing code into different files for maintainability
    - Implemeting chatbot endpoint to talk to your data
- **Creating API Endpoints**
    - Building endpoints for file upload, list, and deletion
    - Testing and validating endpoints

- **Building a Streamlit Interface**
    - Creating a user-friendly interface with Streamlit
    - Integrating the RAG chatbot API with the Streamlit app
    - File management features (upload, list, delete)

###What is Retrieval Augmented Generation (RAG)?
RAG is a technique that enhances language models by combining them with a retrieval system. It allows the model to access and utilize external knowledge when generating responses.

The process typically involves:
####Indexing a large corpus of documents

In [1]:
import langchain
print(langchain.__version__)

0.2.12


In [2]:
  MODEL_SERVICE = "http://localhost:33665/v1"
  API_KEY = "sk-no-key-required"
  

###Call LLM

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(base_url=MODEL_SERVICE, api_key=API_KEY, streaming=True, max_tokens=4096)
llm_response = llm.invoke("what is IBM  watson")

llm_response

AIMessage(content='IBM Watson is a question-answering system developed by IBM. It can process and respond to a wide variety of questions, offering information, providing insights, and performing tasks.', response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo'}, id='run-c6a83c7a-012e-45d3-b613-039ce6ee5d85-0')

###Parsing Output

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(llm_response)

'IBM Watson is a question-answering system developed by IBM. It can process and respond to a wide variety of questions, offering information, providing insights, and performing tasks.'

###Simple Chain

In [8]:
chain = llm | output_parser
chain.invoke("what are IBM watson AI models")

'IBM Watson AI models are a set of advanced artificial intelligence technologies designed to understand and respond to a wide range of user queries with accuracy, relevance, and contextual understanding. These models can be employed in various applications, such as customer service, data analysis, and decision-making, to provide valuable insights and automate complex tasks. Some popular IBM Watson AI models include language translators, sentiment analyzers, and chatbots.'

###Prompt Template

In [12]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("Write a brief summary about {topic}")
prompt.invoke({"topic": "RHEL AI"})

ChatPromptValue(messages=[HumanMessage(content='Write a brief summary about RHEL AI')])

In [13]:
chain = prompt | llm | output_parser
chain.invoke({"topic": "RHEL AI"})

'RHEL AI is a cutting-edge artificial intelligence platform developed by Red Hat, a leading company in the open-source software space. This platform leverages advanced machine learning algorithms to deliver intelligent automation and decision-making capabilities, enabling businesses to streamline their operations, reduce costs, and improve efficiency. RHEL AI is built on top of Red Hat Enterprise Linux (RHEL), a robust and secure Linux distribution, ensuring seamless integration with existing IT infrastructures. With RHEL AI, businesses can unlock valuable insights from their data, automate complex tasks, and make informed decisions based on real-time analytics. This technology has wide-ranging applications across various industries, including finance, healthcare, manufacturing, and retail, making it an essential tool for organizations seeking to stay competitive in the digital age.'

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define the prompt
prompt = ChatPromptTemplate.from_template("describe me  Granite teacher student model about {topic}")

# Initialize the LLM
llm = ChatOpenAI(base_url=MODEL_SERVICE, api_key=API_KEY, streaming=True, max_tokens=4096)

# Define the output parser
output_parser = StrOutputParser()

# Compose the chain
chain = prompt | llm | output_parser

# Use the chain
result = chain.invoke({"topic": "Instructlab"})
print(result)

InstructLab is an innovative educational technology company that specializes in creating interactive and engaging learning experiences for students and educators. Our mission is to empower learners and facilitate their growth through the development of cutting-edge tools and resources designed to enhance teaching and learning. InstructLab's team comprises experienced educators, software developers, and designers who collaborate to bring the most effective educational solutions to life.

One of our flagship products is the InstructLab Virtual Lab, which offers a powerful and flexible virtualization platform for hands-on scientific experimentation in various disciplines, such as physics, chemistry, biology, and engineering. This cloud-based solution allows students to perform realistic simulations, conduct experiments remotely, and collaborate with their peers from anywhere in the world. The InstructLab Virtual Lab is highly scalable and adaptable, making it suitable for use in K-12 clas

###LLM Messages

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="You are a helpful assistant that provides knowledge.")
human_message = HumanMessage(content="Tell me about AI programming")
llm.invoke([system_message, human_message])

AIMessage(content='Artificial Intelligence (AI) programming is a subfield of computer science that focuses on creating algorithms and systems capable of performing tasks that would normally require human intelligence. This includes understanding natural language, recognizing patterns, solving problems, and making decisions. AI programming involves the development of models and techniques from various areas such as machine learning, deep learning, neural networks, and data mining. The ultimate goal is to build intelligent systems that can learn from experience, adapt to new situations, and improve their performance over time.', response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo'}, id='run-062bd332-74bb-457d-bf26-a29bd1e4bb22-0')

In [17]:
template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that provides knowledge."),
    ("human", "Tell me about {topic}")
])

prompt_value = template.invoke(
    {
        "topic": "programming"
    }
)
prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant that provides knowledge.'), HumanMessage(content='Tell me about programming')])

In [18]:
llm.invoke(prompt_value)

AIMessage(content="Programming is a versatile and essential skill in today's digital age. It involves creating, designing, and maintaining computer software, applications, or systems. Programmers use various programming languages to write code that enables computers to perform specific tasks or solve complex problems.\n\nThere are several programming paradigms, including procedural, object-oriented, functional, and logic-based programming. Each paradigm has its own set of rules and best practices, and understanding them can help you choose the most suitable approach for a given task.\n\nProgramming languages have their unique features, syntax, and use cases. Some popular programming languages include Python, Java, C++, JavaScript, C#, and Rust. Each language has its own strengths and weaknesses, and selecting the right one for a specific project can make a significant difference in terms of efficiency, maintainability, and performance.\n\nIn addition to technical skills, programmers ne

In [ ]:
# !pip install docx2txt pypdf unstructured

In [43]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from typing import List
from langchain_core.documents import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

docx_loader = Docx2txtLoader("/home/kkakarla/Development/git-projects/RAG/Langchain-RAG-chatbot/docs/IBM_Watson_History.docx")
documents = docx_loader.load()

print(len(documents))

splits = text_splitter.split_documents(documents)

print(f"Split the documents into {len(splits)} chunks.")

1
Split the documents into 4 chunks.


In [37]:
documents[0]

Document(metadata={'source': '/home/kkakarla/Development/git-projects/RAG/Langchain-RAG-chatbot/docs/IBM_Watson_History.docx'}, page_content="IBM’s research in AI goes back to the 1950s and includes significant milestones like the supercomputer Deep Blue defeating chess grandmaster Garry Kasparov.\xa0\n\n\n\nIn 2011, IBM Watson defeated Brad Rutter and Ken Jennings in the\xa0Jeopardy!\xa0Challenge.\xa0To find and understand the clues in the questions, Watson compared possible answers by ranking its confidence in their accuracy, and responded —\xa0all in under three seconds.\n\n\n\nWatson sparked curiosity around “machines that could think” and opened up the possibilities around how AI could be applied to business. Clients in industries ranging from financial services to retail put Watson to work to unlock new insights, drive productivity and deliver better customer experiences.\xa0\n\n\n\nNow through advancements in core Watson technologies, IBM has developed the next generation AI and

In [38]:
splits[1]

Document(metadata={'source': '/home/kkakarla/Development/git-projects/RAG/Langchain-RAG-chatbot/docs/IBM_Watson_History.docx'}, page_content='How Watson has evolved AI over time\n\n\n\n2007\n\nIBM Watson\n\nIBM Research started working on the grand challenge of building a computer system that could compete with champions at the game of Jeopardy!. Just four years later in 2011, the open-domain question-answering system dubbed Watson beat the two highest ranked players in a nationally televised two-game Jeopardy! Match.\n\n\n\n2013\n\nIBM Watson Developer Cloud\n\nIBM Watson technology became available as a development platform in the cloud. The move spurred innovation and fueled a new ecosystem of entrepreneurial software application providers–ranging from start-ups and emerging, venture capital-backed businesses to established players.\n\n\n\n\n\n2014\n\nIBM Watson Discovery Advisor\n\nIBM Watson Discovery Advisor was designed to cut through data quickly and find unexpected connections

In [39]:
splits[0].metadata

{'source': '/home/kkakarla/Development/git-projects/RAG/Langchain-RAG-chatbot/docs/IBM_Watson_History.docx'}

In [40]:
splits[0].page_content

'IBM’s research in AI goes back to the 1950s and includes significant milestones like the supercomputer Deep Blue defeating chess grandmaster Garry Kasparov.\xa0\n\n\n\nIn 2011, IBM Watson defeated Brad Rutter and Ken Jennings in the\xa0Jeopardy!\xa0Challenge.\xa0To find and understand the clues in the questions, Watson compared possible answers by ranking its confidence in their accuracy, and responded —\xa0all in under three seconds.\n\n\n\nWatson sparked curiosity around “machines that could think” and opened up the possibilities around how AI could be applied to business. Clients in industries ranging from financial services to retail put Watson to work to unlock new insights, drive productivity and deliver better customer experiences.\xa0\n\n\n\nNow through advancements in core Watson technologies, IBM has developed the next generation AI and data platform and set of AI assistants with\xa0watsonx.\n\n\n\nHow Watson has evolved AI over time\n\n\n\n2007\n\nIBM Watson'

In [41]:
# 1. Function to load documents from a folder
import os
def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

# Load documents from a folder
folder_path = "/home/kkakarla/Development/git-projects/RAG/Langchain-RAG-chatbot/docs"
documents = load_documents(folder_path)

print(f"Loaded {len(documents)} documents from the folder.")
splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")

Loaded 3 documents from the folder.
Split the documents into 19 chunks.


In [44]:
embeddings = HuggingFaceEmbeddings()

# 4. Embedding Documents

document_embeddings = embeddings.embed_documents([split.page_content for split in splits])

print(f"Created embeddings for {len(document_embeddings)} document chunks.")

/home/kkakarla/Development/git-projects/venv/lib64/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/kkakarla/Development/git-projects/venv/lib64/python3.11/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Created embeddings for 4 document chunks.


In [45]:
document_embeddings[0]

[0.0648818090558052,
 0.018800826743245125,
 -0.046932972967624664,
 0.006365220993757248,
 -0.04207936301827431,
 -0.012168326415121555,
 0.017815927043557167,
 0.006736586801707745,
 0.027553094550967216,
 -0.022305235266685486,
 0.03175883740186691,
 0.057454586029052734,
 -0.019073404371738434,
 0.05308182165026665,
 0.014643300324678421,
 -0.04887054115533829,
 0.029301609843969345,
 -0.00017492080223746598,
 0.019882511347532272,
 0.006077866069972515,
 -0.017813818529248238,
 -0.010979913175106049,
 -0.03378232568502426,
 0.010563940741121769,
 -0.027939926832914352,
 -0.0036256099119782448,
 -0.002171413041651249,
 -0.014232911169528961,
 0.006182192824780941,
 -0.02790074795484543,
 -0.023442523553967476,
 -0.008620800450444221,
 -0.009038279764354229,
 0.038681890815496445,
 2.184052846132545e-06,
 -0.0643359124660492,
 0.010319344699382782,
 -0.0019997279159724712,
 -0.07483778148889542,
 -0.018127772957086563,
 -0.035964127629995346,
 0.061379965394735336,
 0.00345143512822

In [46]:
# !pip install sentence_transformers
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
document_embeddings[0]

/home/kkakarla/Development/git-projects/venv/lib64/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


[-0.08797682076692581,
 -0.05038750171661377,
 0.025851113721728325,
 0.0027760090306401253,
 -0.03903089091181755,
 0.08132871240377426,
 0.022626835852861404,
 0.0735144168138504,
 0.020771879702806473,
 -0.010132277384400368,
 -0.06287723779678345,
 0.041981857270002365,
 0.09130679816007614,
 -0.018540509045124054,
 0.0008228500955738127,
 0.06177208572626114,
 0.06127754971385002,
 -0.08280260115861893,
 -0.07051585614681244,
 -0.10585687309503555,
 -0.047503456473350525,
 -0.014568772166967392,
 0.022720186039805412,
 -0.06251261383295059,
 0.03204067423939705,
 0.02700805850327015,
 0.01725250668823719,
 -0.058455631136894226,
 -0.01706349477171898,
 0.019619479775428772,
 0.008090189658105373,
 -0.037836626172065735,
 0.02932380512356758,
 0.08000637590885162,
 -0.047701336443424225,
 -0.003315580543130636,
 -0.019049279391765594,
 -0.027199089527130127,
 0.058563437312841415,
 -0.05022890120744705,
 -0.0808422714471817,
 -0.07360894978046417,
 -0.054800037294626236,
 -0.031689

In [ ]:
# !pip install langchain_chroma -q

###Create and persist Chroma vector store

In [47]:
from langchain_chroma import Chroma

embedding_function = HuggingFaceEmbeddings()
collection_name = "my_collection"
vectorstore = Chroma.from_documents(collection_name=collection_name, documents=splits, embedding=embedding_function, persist_directory="./chroma_db")
#db.persist()

print("Vector store created and persisted to './chroma_db'")

Vector store created and persisted to './chroma_db'


In [48]:
# 5. Perform similarity search

query = "When was IBM Watson founded?"
search_results = vectorstore.similarity_search(query, k=2)

print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()


Top 2 most relevant chunks for the query: 'When was IBM Watson founded?'

Result 1:
Source: /home/kkakarla/Development/git-projects/RAG/Langchain-RAG-chatbot/docs/IBM_Watson_History.docx
Content: IBM’s research in AI goes back to the 1950s and includes significant milestones like the supercomputer Deep Blue defeating chess grandmaster Garry Kasparov. 



In 2011, IBM Watson defeated Brad Rutter and Ken Jennings in the Jeopardy! Challenge. To find and understand the clues in the questions, Watson compared possible answers by ranking its confidence in their accuracy, and responded — all in under three seconds.



Watson sparked curiosity around “machines that could think” and opened up the possibilities around how AI could be applied to business. Clients in industries ranging from financial services to retail put Watson to work to unlock new insights, drive productivity and deliver better customer experiences. 



Now through advancements in core Watson technologies, IBM has developed t

In [49]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever.invoke("When was IBM Watson founded?")

[Document(metadata={'source': '/home/kkakarla/Development/git-projects/RAG/Langchain-RAG-chatbot/docs/IBM_Watson_History.docx'}, page_content='IBM’s research in AI goes back to the 1950s and includes significant milestones like the supercomputer Deep Blue defeating chess grandmaster Garry Kasparov.\xa0\n\n\n\nIn 2011, IBM Watson defeated Brad Rutter and Ken Jennings in the\xa0Jeopardy!\xa0Challenge.\xa0To find and understand the clues in the questions, Watson compared possible answers by ranking its confidence in their accuracy, and responded —\xa0all in under three seconds.\n\n\n\nWatson sparked curiosity around “machines that could think” and opened up the possibilities around how AI could be applied to business. Clients in industries ranging from financial services to retail put Watson to work to unlock new insights, drive productivity and deliver better customer experiences.\xa0\n\n\n\nNow through advancements in core Watson technologies, IBM has developed the next generation AI an

In [50]:
from langchain_core.prompts import ChatPromptTemplate
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(template)

In [51]:
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt
)
rag_chain.invoke("When was IBM Watson founded?")

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\n[Document(metadata={'source': '/home/kkakarla/Development/git-projects/RAG/Langchain-RAG-chatbot/docs/IBM_Watson_History.docx'}, page_content='IBM’s research in AI goes back to the 1950s and includes significant milestones like the supercomputer Deep Blue defeating chess grandmaster Garry Kasparov.\\xa0\\n\\n\\n\\nIn 2011, IBM Watson defeated Brad Rutter and Ken Jennings in the\\xa0Jeopardy!\\xa0Challenge.\\xa0To find and understand the clues in the questions, Watson compared possible answers by ranking its confidence in their accuracy, and responded —\\xa0all in under three seconds.\\n\\n\\n\\nWatson sparked curiosity around “machines that could think” and opened up the possibilities around how AI could be applied to business. Clients in industries ranging from financial services to retail put Watson to work to unlock new insights, drive productivity and deliver better customer ex

In [52]:
def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [53]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()} | prompt
)
rag_chain.invoke("When was IBM Watson founded?")

ChatPromptValue(messages=[HumanMessage(content='Answer the question based only on the following context:\nIBM’s research in AI goes back to the 1950s and includes significant milestones like the supercomputer Deep Blue defeating chess grandmaster Garry Kasparov.\xa0\n\n\n\nIn 2011, IBM Watson defeated Brad Rutter and Ken Jennings in the\xa0Jeopardy!\xa0Challenge.\xa0To find and understand the clues in the questions, Watson compared possible answers by ranking its confidence in their accuracy, and responded —\xa0all in under three seconds.\n\n\n\nWatson sparked curiosity around “machines that could think” and opened up the possibilities around how AI could be applied to business. Clients in industries ranging from financial services to retail put Watson to work to unlock new insights, drive productivity and deliver better customer experiences.\xa0\n\n\n\nNow through advancements in core Watson technologies, IBM has developed the next generation AI and data platform and set of AI assista

In [54]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
question = "When was IBM Watson founded?"
response = rag_chain.invoke(question)
print(response)

The IBM Watson NLP Library was founded in 2017. However, based on the context provided, the specific milestone you're asking about is when IBM Research started working on building a computer system that could compete with champions at the game of Jeopardy!, which occurred in 2007.


###Conversational RAG

####Handling Follow Up Questions

In [55]:
# Example conversation
from langchain_core.messages import HumanMessage, AIMessage
chat_history = []
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response)
])

In [56]:
chat_history

[HumanMessage(content='When was IBM Watson founded?'),
 AIMessage(content="The IBM Watson NLP Library was founded in 2017. However, based on the context provided, the specific milestone you're asking about is when IBM Research started working on building a computer system that could compete with champions at the game of Jeopardy!, which occurred in 2007.")]

In [57]:
from langchain_core.prompts import MessagesPlaceholder
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# history_aware_retriever = create_history_aware_retriever(
#     llm, retriever, contextualize_q_prompt
# )
contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
contextualize_chain.invoke({"input": "Where it is headquartered?", "chat_history": chat_history})

"The IBM Watson headquarters is located in Armonk, New York, United States. To provide more context, Armonk is a suburb north of New York City and serves as the corporate hub for IBM's global operations."

In [58]:
from langchain.chains import create_history_aware_retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)
history_aware_retriever.invoke({"input": "Where it is headquartered?", "chat_history": chat_history})

[Document(metadata={'source': '/home/kkakarla/Development/git-projects/RAG/Langchain-RAG-chatbot/docs/IBM_Watson_History.docx'}, page_content='IBM’s research in AI goes back to the 1950s and includes significant milestones like the supercomputer Deep Blue defeating chess grandmaster Garry Kasparov.\xa0\n\n\n\nIn 2011, IBM Watson defeated Brad Rutter and Ken Jennings in the\xa0Jeopardy!\xa0Challenge.\xa0To find and understand the clues in the questions, Watson compared possible answers by ranking its confidence in their accuracy, and responded —\xa0all in under three seconds.\n\n\n\nWatson sparked curiosity around “machines that could think” and opened up the possibilities around how AI could be applied to business. Clients in industries ranging from financial services to retail put Watson to work to unlock new insights, drive productivity and deliver better customer experiences.\xa0\n\n\n\nNow through advancements in core Watson technologies, IBM has developed the next generation AI an

In [59]:
retriever.invoke("Where it is headquartered?")

[Document(metadata={'source': '/home/kkakarla/Development/git-projects/RAG/Langchain-RAG-chatbot/docs/IBM_Watson_History.docx'}, page_content='2023\n\nwatsonx\n\nIn 2023, IBM announced the watsonx platform, which allows partners to train, tune and distribute models with generative AI and machine learning capabilities. Under development for three years, IBM designed watsonx to manage the life cycle of foundation models that are the basis of generative AI capabilities and for creating and tuning machine learning models'),
 Document(metadata={'source': '/home/kkakarla/Development/git-projects/RAG/Langchain-RAG-chatbot/docs/IBM_Watson_History.docx'}, page_content='IBM’s research in AI goes back to the 1950s and includes significant milestones like the supercomputer Deep Blue defeating chess grandmaster Garry Kasparov.\xa0\n\n\n\nIn 2011, IBM Watson defeated Brad Rutter and Ken Jennings in the\xa0Jeopardy!\xa0Challenge.\xa0To find and understand the clues in the questions, Watson compared p

In [60]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [61]:
rag_chain.invoke({"input": "Where it is headquartered?", "chat_history":chat_history})

{'input': 'Where it is headquartered?',
 'chat_history': [HumanMessage(content='When was IBM Watson founded?'),
  AIMessage(content="The IBM Watson NLP Library was founded in 2017. However, based on the context provided, the specific milestone you're asking about is when IBM Research started working on building a computer system that could compete with champions at the game of Jeopardy!, which occurred in 2007.")],
 'context': [Document(metadata={'source': '/home/kkakarla/Development/git-projects/RAG/Langchain-RAG-chatbot/docs/IBM_Watson_History.docx'}, page_content='IBM’s research in AI goes back to the 1950s and includes significant milestones like the supercomputer Deep Blue defeating chess grandmaster Garry Kasparov.\xa0\n\n\n\nIn 2011, IBM Watson defeated Brad Rutter and Ken Jennings in the\xa0Jeopardy!\xa0Challenge.\xa0To find and understand the clues in the questions, Watson compared possible answers by ranking its confidence in their accuracy, and responded —\xa0all in under th

###Building Multi User Chatbot

In [62]:
import sqlite3
from datetime import datetime

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
                    (id INTEGER PRIMARY KEY AUTOINCREMENT,
                     session_id TEXT,
                     user_query TEXT,
                     gpt_response TEXT,
                     model TEXT,
                     created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()

In [63]:
import uuid
session_id = str(uuid.uuid4())
chat_history = get_chat_history(session_id)
print(chat_history)
question1 = "When was IBM Watson founded?"
answer1 = rag_chain.invoke({"input": question1, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question1, answer1, "granite-7b-lab")
print(f"Human: {question1}")
print(f"AI: {answer1}\n")

[]
Human: When was IBM Watson founded?
AI: The IBM Watson brand was established in 2011, following the acquisition of Watson Health by IBM. However, the roots of the underlying technology can be traced back to earlier research and development efforts at IBM. The IBM Watson brand was launched in 2014, during which time the company expanded its offerings to include various AI-powered solutions and services. This timeline reflects the evolution of IBM Watson from its initial inception to its current status as a prominent player in the field of artificial intelligence and machine learning.



In [64]:
question2 = "Where it is headquartered?"
chat_history = get_chat_history(session_id)
print(chat_history)
answer2 = rag_chain.invoke({"input": question2, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "granite-7b-lab")
print(f"Human: {question2}")
print(f"AI: {answer2}\n")

[{'role': 'human', 'content': 'When was IBM Watson founded?'}, {'role': 'ai', 'content': 'The IBM Watson brand was established in 2011, following the acquisition of Watson Health by IBM. However, the roots of the underlying technology can be traced back to earlier research and development efforts at IBM. The IBM Watson brand was launched in 2014, during which time the company expanded its offerings to include various AI-powered solutions and services. This timeline reflects the evolution of IBM Watson from its initial inception to its current status as a prominent player in the field of artificial intelligence and machine learning.'}]
Human: Where it is headquartered?
AI: The IBM Watson headquarters is located in Armonk, New York. Armonk is a suburb north of New York City, and the campus is situated near the intersection of Route 1 and the Saw Mill River Parkway. This location provides easy access to major highways and airports, making it an ideal site for IBM's global headquarters. Th

New User

In [65]:
session_id = str(uuid.uuid4())
question = "What is IBM Watson"
chat_history = get_chat_history(session_id)
print(chat_history)
answer = rag_chain.invoke({"input": question, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question, answer, "granite-7b-lab")
print(f"Human: {question}")
print(f"AI: {answer}\n")

[]
Human: What is IBM Watson
AI: IBM Watson is a powerful artificial intelligence (AI) platform developed by IBM. It offers a wide range of services and tools designed to help businesses automate complex tasks, analyze large data sets, and make informed decisions based on the insights gleaned from that data. Some of the key capabilities of IBM Watson include natural language processing, machine learning, computer vision, and automation. These features enable Watson to understand and respond to human language, recognize patterns in images and videos, and automate repetitive tasks, making it a versatile and powerful tool for businesses across industries. Additionally, IBM Watson integrates with a wide range of third-party applications and platforms, allowing businesses to leverage its capabilities within their existing workflows and systems.

